## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cidreira,BR,-30.1811,-50.2056,78.26,86,100,7.40,overcast clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,55.40,58,75,2.30,broken clouds
2,2,Castro,BR,-24.7911,-50.0119,72.79,79,100,8.46,overcast clouds
3,3,Cockburn Town,TC,21.4612,-71.1419,78.80,53,40,12.66,scattered clouds
4,4,Sakaiminato,JP,35.5500,133.2333,39.20,75,75,3.44,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.tail(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
687,687,Sinnamary,GF,5.3833,-52.9500,79.47,77,32,14.92,scattered clouds
688,688,Beloha,MG,-25.1667,45.0500,77.95,78,49,3.58,scattered clouds
689,689,Loa Janan,ID,-0.5830,117.0950,72.61,98,100,2.66,overcast clouds
690,690,Gigmoto,PH,13.7796,124.3900,76.33,83,97,13.96,overcast clouds
691,691,Jawhar,SO,2.7809,45.5005,72.21,79,0,15.90,clear sky
692,692,Maracaibo,VE,10.6317,-71.6406,87.08,54,39,8.30,scattered clouds
693,693,Kuah,MY,6.3265,99.8432,80.60,69,20,2.30,few clouds
695,695,Sokoto,NG,13.0609,5.2390,76.30,17,0,7.76,clear sky
698,698,Quelimane,MZ,-17.8786,36.8883,79.11,87,0,4.85,clear sky
699,699,Aripuana,BR,-9.1667,-60.6333,89.69,56,49,2.28,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                261
City                   261
Country                258
Lat                    261
Lng                    261
Max Temp               261
Humidity               261
Cloudiness             261
Wind Speed             261
Current Description    261
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df_clean = preferred_cities_df.dropna()
preferred_cities_df_clean.count()

City_ID                258
City                   258
Country                258
Lat                    258
Lng                    258
Max Temp               258
Humidity               258
Cloudiness             258
Wind Speed             258
Current Description    258
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df_clean[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cidreira,BR,78.26,overcast clouds,-30.1811,-50.2056,
2,Castro,BR,72.79,overcast clouds,-24.7911,-50.0119,
3,Cockburn Town,TC,78.80,scattered clouds,21.4612,-71.1419,
6,Salalah,OM,71.60,clear sky,17.0151,54.0924,
8,Dong Xoai,VN,70.21,few clouds,11.5333,106.9167,
12,Namibe,AO,74.17,overcast clouds,-15.1961,12.1522,
13,Cape Town,ZA,70.00,light rain,-33.9258,18.4232,
14,Matara,LK,75.15,clear sky,5.9485,80.5353,
15,Rikitea,PF,78.93,few clouds,-23.1203,-134.9692,
16,Diego De Almagro,CL,84.13,clear sky,-26.3667,-70.0500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df.count()

City                   258
Country                258
Max Temp               258
Current Description    258
Lat                    258
Lng                    258
Hotel Name             258
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))